In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

from ib_insync import util, IB
util.startLoop()

# Set the root
from from_root import from_root
ROOT = from_root()

# set vscode path
import sys
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>

In [ ]:
import asyncio

from src.nse import Vars, create_target_opts
from src.utils import (Vars, get_pickle, 
                       chunk_me, flatten,
                       get_prec, pickle_with_age_check,)

from tqdm.asyncio import tqdm
import pandas as pd

MARKET = 'NSE'
_vars = Vars(MARKET)
MINEXPROM = _vars.MINEXPROM
PORT = _vars.PORT

In [ ]:
all_opts_path = ROOT / 'data' / 'nse' / 'df_all_opts.pkl'

opt_prices_path = ROOT / 'data' / MARKET / 'df_opt_prices.pkl'
opt_margins_path = ROOT / 'data' / MARKET / 'df_opt_margins.pkl'

# df_all_opts = get_pickle(all_opts_path)
df_opt_prices = get_pickle(opt_prices_path)
df_opt_margins = get_pickle(opt_margins_path)

In [ ]:
# Get alll the options
df_all_opts = create_target_opts(df_opt_prices, 
                                    df_opt_margins, 
                                    _vars.MINEXPROM)



In [ ]:
lots = get_pickle(ROOT / 'data' / 'nse' / 'lots.pkl')

In [ ]:
cols = [x for x in list(df_opt_margins) if x not in list(df_opt_prices)]
df_all_opts = pd.concat([df_opt_prices, df_opt_margins[cols]], axis=1)

# Get precise expected prices
df_all_opts = df_all_opts.assign(expPrice = df_all_opts.apply(lambda x: 
                                get_prec(((MINEXPROM*x.dte/365*x.margin)+x.comm)
                                        /x.lot_size, 0.05), 
                                            axis=1))

In [ ]:
df_all_opts